In [1]:
import os
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv()

GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL = os.getenv("GITHUB_MODEL")

if not all([GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL]):
    raise RuntimeError(
        "❌ Faltan variables en .env. Asegúrate de tener "
        "GITHUB_ENDPOINT, GITHUB_TOKEN y GITHUB_MODEL configuradas."
    )

# Adaptar a las variables esperadas por OpenAIChatClient
os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

print("✅ Configuración cargada desde .env")
print("  ENDPOINT:", GITHUB_ENDPOINT)
print("  MODEL:", GITHUB_MODEL)


✅ Configuración cargada desde .env
  ENDPOINT: https://models.github.ai/inference
  MODEL: gpt-4o


In [3]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

TICKETS_CSV = DATA_DIR / "tickets.csv"

# Si el fichero no existe, lo inicializamos con cabecera
if not TICKETS_CSV.exists():
    df_init = pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )
    df_init.to_csv(TICKETS_CSV, index=False, encoding="utf-8")
    print(f"✅ Fichero de tickets creado: {TICKETS_CSV}")
else:
    print(f"✅ Fichero de tickets encontrado: {TICKETS_CSV}")

def load_tickets() -> pd.DataFrame:
    """
    Carga el CSV de tickets en un DataFrame de pandas.
    Si no existe o está vacío, devuelve un DataFrame con las columnas estándar.
    """
    if TICKETS_CSV.exists():
        return pd.read_csv(TICKETS_CSV, dtype=str).fillna("")
    return pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )

# Vista rápida para comprobar
df_preview = load_tickets()
print("🔎 Vista previa de tickets.csv:")
display(df_preview.head())



✅ Fichero de tickets encontrado: data\tickets.csv
🔎 Vista previa de tickets.csv:


,id,fecha,solicitante,departamento,categoria,prioridad,estado,resumen,detalle
0,1,2025-01-10,antonio.soto@empresa.local,IT,nuevo_equipo,alta,pendiente,Solicitud portátil para teletrabajo,Necesito un portátil con al menos 16GB de RAM ...
1,2,2025-01-12,maria.fraga@empresa.local,RRHH,certificado,media,resuelto_auto,Certificado de empresa,Solicito un certificado de empresa para presen...
2,3,2025-01-15,carlos.gomez@empresa.local,IT,incidencia,alta,en_progreso,VPN no funciona,No puedo conectar a la VPN corporativa desde m...
3,4,2025-01-18,laura.fernandez@empresa.local,Facilities,mantenimiento,media,pendiente,Problema con aire acondicionado,En la sala de reuniones 3A el aire acondiciona...
4,5,2025-01-20,antonio.soto@empresa.local,RRHH,vacaciones,baja,resuelto_auto,Consulta sobre días de vacaciones,Quisiera confirmar cuántos días de vacaciones ...


In [6]:
from typing import Dict,Annotated
from pydantic import Field

def crear_ticket_fc(
    desde_email: Annotated[str, Field(description="Dirección de correo del solicitante")],
    departamento: Annotated[str, Field(description="Departamento destino: IT, RRHH, Facilities u otro")],
    categoria: Annotated[str, Field(description="Categoria de la solicitud: nuevo_equipo, incidencia, vacaciones, certificado, mantenimiento, otro")],
    prioridad: Annotated[str, Field(description="Prioridad: alta, media o baja")],
    resumen: Annotated[str, Field(description="Resumen corto de la solicitud")],
    detalle: Annotated[str, Field(description="Detalle extendido de la solicitud")]
) -> Dict:
    """
    Crea un nuevo ticket en tickets.csv y guarda un JSON en out/ticket_{id}.json.
    Devuelve el diccionario con los datos completos del ticket creado.
    """
    data = load_tickets()

    if data.empty:
        next_id = 1
    else:
        ids = pd.to_numeric(data["id"], errors="coerce")
        next_id = int(ids.max() or 0) + 1

    nuevo_ticket = {
        "id": str(next_id),
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "solicitante": desde_email,
        "departamento": departamento or "Otro",
        "categoria": categoria or "otro",
        "prioridad": prioridad or "media",
        "estado": "pendiente",
        "resumen": (resumen or "")[:200],
        "detalle": detalle or "",
    }

    data = pd.concat([data, pd.DataFrame([nuevo_ticket])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_json_path = OUT_DIR / f"ticket_{next_id}.json"
    with ticket_json_path.open("w", encoding="utf-8") as f:
        json.dump(nuevo_ticket, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Ticket creado con id={next_id}")
    return nuevo_ticket


def actualizar_estado_ticket_fc(
    id_ticket: Annotated[int, Field(description="Identificador numérico del ticket a actualizar")],
    nuevo_estado: Annotated[str, Field(description="Nuevo estado del ticket, por ejemplo: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado")]
) -> Dict:
    """
    Actualiza el campo 'estado' de un ticket en tickets.csv y en el JSON correspondiente (si existe).
    Devuelve el ticket actualizado, o un mensaje de error si el id no existe.
    """
    data = load_tickets()
    if data.empty:
        return {
            "ok": False,
            "mensaje": f"No hay tickets en el sistema. No se encontró el ticket con id={id_ticket}."
        }

    mask = data["id"] == str(id_ticket)
    if not mask.any():
        return {
            "ok": False,
            "mensaje": f"No se encontró el ticket con id={id_ticket}."
        }

    data.loc[mask, "estado"] = nuevo_estado
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_actualizado = data.loc[mask].iloc[0].to_dict()

    ticket_json_path = OUT_DIR / f"ticket_{id_ticket}.json"
    if ticket_json_path.exists():
        with ticket_json_path.open("w", encoding="utf-8") as f:
            json.dump(ticket_actualizado, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Estado del ticket {id_ticket} actualizado a '{nuevo_estado}'")
    return {
        "ok": True,
        "ticket": ticket_actualizado
    }


In [8]:
import json

PROFILES_JSON = DATA_DIR / "profiles.json"

# Si no existe, lo inicializamos como dict vacío
if not PROFILES_JSON.exists():
    with PROFILES_JSON.open("w", encoding="utf-8") as f:
        json.dump({}, f, ensure_ascii=False, indent=2)
    print(f"✅ Fichero de perfiles creado: {PROFILES_JSON}")
else:
    print(f"✅ Fichero de perfiles encontrado: {PROFILES_JSON}")

def load_all_profiles() -> Dict[str, Dict]:
    """Carga todos los perfiles desde profiles.json."""
    if PROFILES_JSON.exists():
        with PROFILES_JSON.open("r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_all_profiles(profiles: Dict[str, Dict]) -> None:
    """Guarda todos los perfiles en profiles.json."""
    with PROFILES_JSON.open("w", encoding="utf-8") as f:
        json.dump(profiles, f, ensure_ascii=False, indent=2)

def get_user_profile(email: str) -> Dict:
    """Devuelve el perfil de un usuario (creando uno básico si no existe)."""
    profiles = load_all_profiles()
    email_key = email.lower()
    if email_key not in profiles:
        profiles[email_key] = {
            "email": email_key,
            "sede": "",
            "preferencias_equipo": "",
            "idioma_respuesta": "es"
        }
        save_all_profiles(profiles)
    return profiles[email_key]

def update_user_profile(email: str, sede: str = "", preferencias_equipo: str = "", idioma_respuesta: str = "") -> Dict:
    """Actualiza campos de perfil (solo los no vacíos) y devuelve el perfil actualizado."""
    profiles = load_all_profiles()
    email_key = email.lower()
    perfil = profiles.get(email_key, {
        "email": email_key,
        "sede": "",
        "preferencias_equipo": "",
        "idioma_respuesta": "es"
    })

    if sede:
        perfil["sede"] = sede
    if preferencias_equipo:
        perfil["preferencias_equipo"] = preferencias_equipo
    if idioma_respuesta:
        perfil["idioma_respuesta"] = idioma_respuesta

    profiles[email_key] = perfil
    save_all_profiles(profiles)
    return perfil

# Prueba rápida
p = update_user_profile("antonio.soto@empresa.local", sede="Barcelona", preferencias_equipo="Lenovo")
print("Perfil de prueba:", p)


✅ Fichero de perfiles encontrado: data\profiles.json
Perfil de prueba: {'preferencias_equipo': 'Lenovo', 'sede': 'Barcelona'}


In [9]:
def actualizar_perfil_usuario_fc(
    email: Annotated[str, Field(description="Email corporativo del usuario para guardar su perfil")],
    sede: Annotated[str, Field(description="Ciudad o sede principal de trabajo del usuario")] = "",
    preferencias_equipo: Annotated[str, Field(description="Preferencia de marca o tipo de portátil (p.ej. Lenovo, Dell, etc.)")] = "",
    idioma_respuesta: Annotated[str, Field(description="Idioma preferido para las respuestas (es, en, etc.)")] = ""
) -> Dict:
    """
    Actualiza el perfil de usuario (sede, preferencias de equipo, idioma) en profiles.json.
    Devuelve el perfil completo después de la actualización.
    """
    perfil = update_user_profile(
        email=email,
        sede=sede,
        preferencias_equipo=preferencias_equipo,
        idioma_respuesta=idioma_respuesta
    )
    print(f"✅ [Tool] Perfil actualizado para {email}: {perfil}")
    return perfil


In [10]:
SERVICE_DESK_MEMORY_INSTRUCTIONS = """
Eres un agente de Service Desk interno con capacidad de recordar preferencias del usuario.

Tu trabajo es:
1. Entender la solicitud del usuario en lenguaje natural.
2. Consultar el perfil del usuario (que se te pasa en el mensaje como JSON) y usarlo como contexto:
   - sede
   - preferencias de portátil (preferencias_equipo)
   - idioma_respuesta
3. Decidir si debe crearse un ticket nuevo o modificarse uno existente.
4. Si procede, llamar a las funciones disponibles:
   - `crear_ticket_fc` para registrar un nuevo ticket.
   - `actualizar_estado_ticket_fc` para cambiar el estado de un ticket existente.
   - `actualizar_perfil_usuario_fc` para actualizar las preferencias del usuario cuando te lo pida.
5. Responder al usuario confirmando la acción realizada (id del ticket, nuevo estado, etc.).

Reglas:
- Si el usuario da información de perfil (“trabajo en Barcelona”, “prefiero portátiles Lenovo”), deberías llamar a `actualizar_perfil_usuario_fc`.
- Cuando crees un ticket, si conoces la sede o preferencia de portátil del usuario, intenta incluirlo en el detalle del ticket.
- Pregunta por el email del usuario si no se ha proporcionado antes de operaciones críticas (crear ticket, guardar perfil).
- No inventes identificadores ni emails.
- Usa estados coherentes: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado, etc.
"""


In [18]:
from agent_framework import ChatAgent

async def create_service_desk_agent_with_memory() -> ChatAgent:
    """
    Agente de Service Desk que:
    - Usa function calling para tickets (crear y actualizar estado).
    - Usa function calling para actualizar el perfil del usuario.
    """
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="ServiceDeskAgentMemory",
        instructions=SERVICE_DESK_MEMORY_INSTRUCTIONS,
        tools=[crear_ticket_fc, actualizar_estado_ticket_fc, actualizar_perfil_usuario_fc],
    )
    return agent


In [13]:
async def run_with_user_profile(agent: ChatAgent, email_usuario: str, texto_usuario: str, thread=None):
    """
    Carga el perfil del usuario, lo inyecta como contexto en el mensaje
    y llama al agente.
    """
    perfil = get_user_profile(email_usuario)
    perfil_json = json.dumps(perfil, ensure_ascii=False)

    # Inyectamos el perfil de forma explícita en el mensaje
    mensaje = (
        f"Perfil actual del usuario (JSON): {perfil_json}\n\n"
        f"Mensaje del usuario: {texto_usuario}"
    )

    if thread is None:
        thread = agent.get_new_thread()

    result = await agent.run(mensaje, thread=thread)
    return result, thread


In [15]:
# El usuario configura su perfil
from agent_framework.openai import OpenAIChatClient

async def demo_configurar_perfil():
    agent = await create_service_desk_agent_with_memory()
    thread = agent.get_new_thread()

    email = "antonio.soto@empresa.local"
    texto = "Quiero que recuerdes que trabajo en Barcelona y que prefiero portátiles Lenovo."

    print(f"👤 Usuario ({email}): {texto}\n")

    result, thread = await run_with_user_profile(agent, email, texto, thread)
    print("🤖 ServiceDeskAgentMemory (respuesta):\n")
    print(result.text)

    # Ver el perfil en disco después de la interacción
    perfil_actual = get_user_profile(email)
    print("\n🧠 Perfil almacenado en profiles.json:\n", perfil_actual)

await demo_configurar_perfil()


👤 Usuario (antonio.soto@empresa.local): Quiero que recuerdes que trabajo en Barcelona y que prefiero portátiles Lenovo.

🤖 ServiceDeskAgentMemory (respuesta):

Actualmente ya tengo registrado en tu perfil tanto la sede "Barcelona" como la preferencia de equipo "Lenovo". No es necesario actualizar tu perfil. ¿Hay algo más en lo que pueda ayudarte?

🧠 Perfil almacenado en profiles.json:
 {'preferencias_equipo': 'Lenovo', 'sede': 'Barcelona'}


In [16]:
# Usamos el perfil

async def demo_crear_ticket_con_memoria():
    agent = await create_service_desk_agent_with_memory()
    thread = agent.get_new_thread()

    email = "antonio.soto@empresa.local"

    # Suponemos que el perfil ya tiene sede=Barcelona y preferencias_equipo=Lenovo
    texto = "Necesito un portátil nuevo, por favor."

    print(f"👤 Usuario ({email}): {texto}\n")

    result, thread = await run_with_user_profile(agent, email, texto, thread)
    print("🤖 ServiceDeskAgentMemory (respuesta):\n")
    print(result.text)

    # Mostramos los tickets de ese usuario para ver qué se ha creado
    data = load_tickets()
    filtrado = data[data["solicitante"].str.lower() == email.lower()]
    print("\n🎫 Tickets del usuario en tickets.csv:")
    display(filtrado.tail(5))

await demo_crear_ticket_con_memoria()


👤 Usuario (antonio.soto@empresa.local): Necesito un portátil nuevo, por favor.

🤖 ServiceDeskAgentMemory (respuesta):

Por favor, indícame tu correo electrónico para poder procesar tu solicitud y crear un ticket para el nuevo portátil.

🎫 Tickets del usuario en tickets.csv:


,id,fecha,solicitante,departamento,categoria,prioridad,estado,resumen,detalle
5,6,2025-11-28,antonio.soto@empresa.local,IT,nuevo_equipo,alta,pendiente,Solicitud de portátil para teletrabajo,Necesito un portátil con 16GB de RAM para trab...
6,7,2025-11-28,antonio.soto@empresa.local,IT,nuevo_equipo,media,pendiente,Solicitud de portátil nuevo con 16GB de RAM,El usuario solicita un portátil nuevo con 16GB...
7,8,2025-11-28,antonio.soto@empresa.local,IT,nuevo_equipo,media,pendiente,Solicitud de portátil con 16GB de RAM,El usuario requiere un portátil nuevo con 16GB...
8,9,2025-11-28,antonio.soto@empresa.local,IT,nuevo_equipo,media,pendiente,Solicitud de portátil con 16GB de RAM para tel...,El usuario solicita un portátil nuevo que cuen...
9,10,2025-11-28,antonio.soto@empresa.local,IT,nuevo_equipo,media,pendiente,Solicitud de portátil nuevo con 16GB de RAM,Necesito un portátil nuevo con 16GB de RAM par...


In [17]:
# Cambiar el estado utilizando memoria

async def demo_actualizar_estado_con_memoria():
    agent = await create_service_desk_agent_with_memory()
    thread = agent.get_new_thread()

    email = "antonio.soto@empresa.local"

    texto = "Cambia el estado del ticket 1 a en_progreso."
    print(f"👤 Usuario ({email}): {texto}\n")

    result, thread = await run_with_user_profile(agent, email, texto, thread)
    print("🤖 ServiceDeskAgentMemory (respuesta):\n")
    print(result.text)

    data = load_tickets()
    print("\n🎫 Tickets actuales:")
    display(data.head())

await demo_actualizar_estado_con_memoria()


👤 Usuario (antonio.soto@empresa.local): Cambia el estado del ticket 1 a en_progreso.

🤖 ServiceDeskAgentMemory (respuesta):

No tengo información sobre tu email para asegurar que eres el usuario asociado al ticket. Por favor, proporcióname tu dirección de correo electrónico para proceder con el cambio de estado del ticket.

🎫 Tickets actuales:


,id,fecha,solicitante,departamento,categoria,prioridad,estado,resumen,detalle
0,1,2025-01-10,antonio.soto@empresa.local,IT,nuevo_equipo,alta,pendiente,Solicitud portátil para teletrabajo,Necesito un portátil con al menos 16GB de RAM ...
1,2,2025-01-12,maria.fraga@empresa.local,RRHH,certificado,media,resuelto_auto,Certificado de empresa,Solicito un certificado de empresa para presen...
2,3,2025-01-15,carlos.gomez@empresa.local,IT,incidencia,alta,en_progreso,VPN no funciona,No puedo conectar a la VPN corporativa desde m...
3,4,2025-01-18,laura.fernandez@empresa.local,Facilities,mantenimiento,media,pendiente,Problema con aire acondicionado,En la sala de reuniones 3A el aire acondiciona...
4,5,2025-01-20,antonio.soto@empresa.local,RRHH,vacaciones,baja,resuelto_auto,Consulta sobre días de vacaciones,Quisiera confirmar cuántos días de vacaciones ...
